## Grabing Public Hotel Occupancy Tax Data, then storing it into a database, crossreferencing if data is repeating

#### Prerequisites:
##### requirements for mysql-python communication:
* pip install mysqlclient
* pip install mysql-connector-python 
* if recieving wheel error: pip install wheel

In [18]:
# Imports
import sys
from zipfile import ZipFile
import pandas as pd
import pandas.io.sql as pdsql
import glob, os
import numpy as np
# Datetime for new column
import datetime
# Imports for mySQL
from sqlalchemy import create_engine, event, DateTime
from db_setup import mysql_user, mysql_password, db_name
import mysql.connector

#### File path defined

In [19]:
mydir = os.path.abspath('./HotelOccupancyTaxData')

In [20]:
mydir

'C:\\DataAnalyticsBootCamp\\WEEK_23 - Project 3\\Project_3_Potential_Marketing\\CRE_Marketing_Data\\HotelOccupancyTaxData'

## Defining headers for data

In [21]:
# Defining header for marketing data. Marketing data comes with no header
ftact_date_head = ['Taxpayer_Number',
                   'Taxpayer_Name',
                   'Taxpayer_Address',
                   'Taxpayer_City',
                   'Taxpayer_State',
                   'Taxpayer_Zip_Code',
                   'Taxpayer_County_Code',
                   'Taxpayer_Organizational_Type',
                   'Taxpayer_Phone_Number',
                   'Record_Type_Code',
                   'Responsibility_Beginning_Date',
                   'Secretary_of_State_File_Number',
                   'SOS_Charter_Date',
                   'SOS_Status_Date',
                   'Current_Exempt_Reason_Code',
                   'Agent_Name',
                   'Agent_Address',
                   'Agent_City',
                   'Agent_State',
                   'Agent_Zip_Code']

ftact_head = ['Taxpayer_Number',
              'Taxpayer_Name',
              'Taxpayer_Address',
              'Taxpayer_City',
              'Taxpayer_State',
              'Taxpayer_Zip_Code',
              'Taxpayer_County_Code',
              'Taxpayer_Organizational_Type',
              'Taxpayer_Phone_Number',
              'Record_Type_Code',
              'Responsibility_Beginning_Date',
              'Responsibility_End_Date',
              'Responsibility_End_Reason_Code',
              'Secretary_of_State_File_Number',
              'SOS_Charter_Date',
              'SOS_Status_Date',
              'SOS_Status_Code',
              'Rigth_to_Tansact_Business_Code',
              'Current_Exempt_Reason_Code',
              'Exempt_Begin_Date',
              'NAICS_Code']

stact_head = ['Taxpayer_Number',
              'Outlet_Number',
              'Taxpayer_Name',
              'Taxpayer_Address',
              'Taxpayer_City',
              'Taxpayer_State',
              'Taxpayer_Zip_Code',
              'Taxpayer_County_Code',
              'Taxpayer_Phone_Number',
              'Taxpayer_Organizational_Type',
              'Outlet_Name',
              'Outlet_Address',
              'Outlet_City',
              'Outlet_State',
              'Outlet_Zip_Code',
              'Outlet_County_Code',
              'Outlet_Phone_Number',
              'Outlet_NAICS_Code',
              'Outlet_Inside_Outside_City_Limits_Indicator',
              'Outlet_Permit_Issue_Date',
              'Outlet_First_Sales_Date']

stact_date_head = ['Taxpayer_Number',
                   'Outlet_Number',
                   'Taxpayer_Name',
                   'Taxpayer_Address',
                   'Taxpayer_City',
                   'Taxpayer_State',
                   'Taxpayer_Zip_Code',
                   'Taxpayer_County_Code',
                   'Taxpayer_Phone_Number',
                   'Outlet_Name',
                   'Outlet_Address',
                   'Outlet_City',
                   'Outlet_State',
                   'Outlet_Zip_Code',
                   'Outlet_County_Code',
                   'Outlet_Phone_Number',
                   'Permit_Type',
                   'Tax_Code',
                   'Outlet_NAICS_Code',
                   'Permit_Issue_Date',
                   'First_Sale_Date']

FTOFFDIR = ['Taxpayer_Number',
            'Sequence_Number',
            'Director_Latest_Year_Filed',
            'Director_Name',
            'Director_Title',
            'Director_Address',
            'Director_City',
            'Director_State',
            'Director_Zip_Code']

## Extract files from zipped folder

In [185]:
# extract all files
i = 0
for file in glob.glob(mydir + '/*.zip'):
    i += 1
    zip = ZipFile(file, 'r')
    print(f'Extracting file {i}')
    zip.extractall(mydir)
    print('Done!')
    print(f"File {i}, extracted: {file}\n")
    zip.close()
    os.remove(file)

Extracting file 1
Done!
File 1, extracted: C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\fran03-16ph.zip

Extracting file 2
Done!
File 2, extracted: C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\stp03-16ph.zip



## Add csv files to a data frame ( fran and stp)

In [186]:
# Searches for a csv file
df_fran = pd.DataFrame()
df_stp = pd.DataFrame()

for file in glob.glob(mydir + '/*.csv'):
    if 'fran' in file:
        df = pd.read_csv(file, header=None, index_col=False, names=ftact_date_head, engine ='python')
        df_fran = df_fran.append(df)
        os.remove(file)
        print('Added the ' + file + " into the DF df_fran")
        print("deleted the file " + str(file))
    elif "stp" in file:
        df = pd.read_csv(file, header=None,index_col=False, names=stact_date_head, engine ='python')
        df_stp = df_stp.append(df)
        os.remove(file)
        print('Added the ' + file + " into the DF df_stp")
        print("deleted the file " + str(file))
    else:
        print('we do not know what to do with this file: ' + str(file))

Added the C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\fran03-16ph.csv into the DF df_fran
deleted the file C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\fran03-16ph.csv
Added the C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\stp03-16ph.csv into the DF df_stp
deleted the file C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\stp03-16ph.csv


#### STP DF created

In [187]:
df_stp.head()

,Taxpayer_Number,Outlet_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Phone_Number,Outlet_Name,...,Outlet_City,Outlet_State,Outlet_Zip_Code,Outlet_County_Code,Outlet_Phone_Number,Permit_Type,Tax_Code,Outlet_NAICS_Code,Permit_Issue_Date,First_Sale_Date
0,32051124918,,"RIVERSIDE HOMEBUILDERS, LTD. ...",3045 LACKLAND RD,FORT WORTH,TX,76116,220,8177177144,...,...,,,,,,DIRECT PAY,27,236115,20200313,20200301
1,32059318793,,"STEELCOAST COMPANY, LLC ...",16200 JOSE GARZA RD,BROWNSVILLE,TX,78521,31,9565092031,...,...,,,,,,DIRECT PAY,27,332811,20200313,20200313
2,32058849996,,"RIVERSIDE (EAST) HOMEBUILDERS, LTD. ...",3045 LACKLAND RD,FORT WORTH,TX,76116,220,8177317595,...,...,,,,,,DIRECT PAY,27,236115,20200313,20200301
3,32073719901,,CESSNA MEXICO S DE R.L. DE C.V. ...,12035 ROJAS DR STE F,EL PASO,TX,79936,71,0 0,...,...,,,,,,MAQUILADORA,28,336412,20200313,20200301
4,14503968167,00001,"APPLIED ENGINEERING, INCORPORATED ...",3300 FIECHTNER DR S C/O CONTROLLER,FARGO,ND,58103,0,7012811884,"APPLIED ENGINEERING, INCORPORATED ...",...,ADDISON,TX,75001,057,7012811884,SALES TAX,26,425120,20200310,20010507


#### FRAN DF created

In [188]:
df_fran.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,Responsibility_Beginning_Date,Secretary_of_State_File_Number,SOS_Charter_Date,SOS_Status_Date,Current_Exempt_Reason_Code,Agent_Name,Agent_Address,Agent_City,Agent_State,Agent_Zip_Code
0,13634731452,GENEVA SCIENTIFIC INC ...,11 N BATAVIA AVE,BATAVIA,IL,60510,0,CF,8003382697,X,20200401,,,,,...,,,,
1,32002687922,"COUNCIL ON LICENSURE, ENFORCEMENT AND REGULATI...",108 WIND HAVEN DR STE A,NICHOLASVILLE,KY,40356,0,CM,8592691401,X,20200408,,,,19,...,,,,
2,32060235911,KICKY PANTS INC ...,179 MADRONE LN N STE 205,BAINBRIDGE ISLAND,WA,98110,0,CF,3605354110,X,20200301,,,,,...,,,,
3,32060765065,"OXFORD INSTRUMENTS ASYLUM RESEARCH, INC ...",6310 HOLLISTER AVE,GOLETA,CA,93117,0,CF,8056966466,X,20200302,,,,,...,,,,
4,32063194149,OLIVIA & OCEAN LLC ...,1701 PECOS VALLEY CV,ROUND ROCK,TX,78665,246,CI,6269220507,X,20200312,,,,,...,,,,


#### Adding the Taxpayer County Name and Record Type Name Column

In [189]:
# Taxpayer Organization Type:

#   CI - Limited Liability Company - Foreign
#   CL - Limited Liability Company - Texas
#   CM - Foreign Non-Profit
#   CN - Texas Non-Profit
#   CP - Professional
#   CR - Texas Insurance
#   CS - Foreign Insurance - OOS
#   CT - Texas Profit
#   CW - Texas Railroad Corporation
#   CX - Foreign Railroad Corporation - OOS
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CF'),'Taxpayer_Organizational_Name']='Foreign Profit'
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CI'),'Taxpayer_Organizational_Name']='Limited Liability Company - Foreign'
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CL'),'Taxpayer_Organizational_Name']='Limited Liability Company - Texas'
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CM'),'Taxpayer_Organizational_Name']='Foreign Non-Profit'
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CN'),'Taxpayer_Organizational_Name']='Texas Non-Profit'
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CP'),'Taxpayer_Organizational_Name']='Professional'
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CR'),'Taxpayer_Organizational_Name']='Texas Insurance'
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CS'),'Taxpayer_Organizational_Name']='Foreign Insurance - OOS'
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CT'),'Taxpayer_Organizational_Name']='Texas Profit'
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CW'),'Taxpayer_Organizational_Name']='Texas Railroad Corporation'
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CX'),'Taxpayer_Organizational_Name']='Foreign Railroad Corporation - OOS'
# Record Type Code:
#   U = Secretary of State (SOS) File Number
#   V = SOS Certificate of Authority (COA) File Number
#   X = Comptroller Assigned File Number
df_fran.loc[(df_fran.Record_Type_Code == 'U'),'Record_Type_Name']='Secretary of State (SOS) File Number'
df_fran.loc[(df_fran.Record_Type_Code == 'V'),'Record_Type_Name']='SOS Certificate of Authority (COA) File Number'
df_fran.loc[(df_fran.Record_Type_Code == 'X'),'Record_Type_Name']='Comptroller Assigned File Number'
df_fran.head()



,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,...,SOS_Charter_Date,SOS_Status_Date,Current_Exempt_Reason_Code,Agent_Name,Agent_Address,Agent_City,Agent_State,Agent_Zip_Code,Taxpayer_Organizational_Name,Record_Type_Name
0,13634731452,GENEVA SCIENTIFIC INC ...,11 N BATAVIA AVE,BATAVIA,IL,60510,0,CF,8003382697,X,...,,,,...,,,,,Foreign Profit,Comptroller Assigned File Number
1,32002687922,"COUNCIL ON LICENSURE, ENFORCEMENT AND REGULATI...",108 WIND HAVEN DR STE A,NICHOLASVILLE,KY,40356,0,CM,8592691401,X,...,,,19,...,,,,,Foreign Non-Profit,Comptroller Assigned File Number
2,32060235911,KICKY PANTS INC ...,179 MADRONE LN N STE 205,BAINBRIDGE ISLAND,WA,98110,0,CF,3605354110,X,...,,,,...,,,,,Foreign Profit,Comptroller Assigned File Number
3,32060765065,"OXFORD INSTRUMENTS ASYLUM RESEARCH, INC ...",6310 HOLLISTER AVE,GOLETA,CA,93117,0,CF,8056966466,X,...,,,,...,,,,,Foreign Profit,Comptroller Assigned File Number
4,32063194149,OLIVIA & OCEAN LLC ...,1701 PECOS VALLEY CV,ROUND ROCK,TX,78665,246,CI,6269220507,X,...,,,,...,,,,,Limited Liability Company - Foreign,Comptroller Assigned File Number


## Extracting textfile and storing into DF (FTOFFDIR, FTACT, STACT)

In [10]:
for file in glob.glob(mydir + '/*.txt'):
    if "FTOFFDIR" in file:
        df_ftoffdir = pd.read_fwf(file,
                                  widths=[11, 3, 4, 50, 10, 40, 20, 2, 5], header=None,
                                  names=FTOFFDIR, index_col=False)  # FTOOB, FTACT
        df_ftoffdir = df_ftoffdir.append(df_ftoffdir)
        os.remove(file)
        print('Added the ' + file + ' into df_ftoffdir')
        print('deleted the file ' + str(file))
    elif "FTOOB" in file or 'FTACT' in file:
        df_ftact = pd.read_fwf(file,
                               widths=[11, 50, 40, 20, 2, 5, 3, 2, 10, 1, 8, 8, 2, 10, 8, 8, 2, 1, 3, 8, 6],
                               header=None,
                               names=ftact_head, index_col=False, engine= 'python')  # FTOOB, FTACT
        df_ftact = df_ftact.append(df_ftact)
        os.remove(file)
        print('Added the ' + file + ' into df_ftact')
        print('deleted the file ' + str(file))

    elif "STACT" in file and 'Layout' not in file:
        df_stact = pd.read_fwf(file,
                               widths=[11, 5, 50, 40, 20, 2, 5, 3, 10, 2, 50, 40, 20, 2, 5, 3, 10, 6, 1, 8, 8],
                               header=None,
                               names=stact_head, index_col=False)  # FTOOB, FTACT
        df_stact = df_stact.append(df_stact)
        os.remove(file)
        print('Added the ' + file + ' into df_stact')
        print("deleted the file " + str(file))
    elif "CPA_SIFT_Description" in file:
        continue
    else:
        os.remove(file)
        print('File not being used: ' + str(file))

Do not know what to do with this file: C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\CPA_SIFT_Description.txt
Added the C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\FTACT.txt into df_ftact
deleted the file C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\FTACT.txt
Added the C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\FTOFFDIR.txt into df_ftoffdir
deleted the file C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\FTOFFDIR.txt
Do not know what to do with this file: C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\STACT Layout.txt
Added the C:\DataAnalyticsBootCamp\WEEK_23 - Project 

#### STACT DF created

In [11]:
df_stact.head()

,Taxpayer_Number,Outlet_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Phone_Number,Taxpayer_Organizational_Type,...,Outlet_Address,Outlet_City,Outlet_State,Outlet_Zip_Code,Outlet_County_Code,Outlet_Phone_Number,Outlet_NAICS_Code,Outlet_Inside_Outside_City_Limits_Indicator,Outlet_Permit_Issue_Date,Outlet_First_Sales_Date
0,10065041799,1,JOHNNY MARSHALL AND PAUL LIBBY,1616 LAKESIDE DR,GARLAND,TX,75042.0,57,9.722721e+09,PI,...,1616 LAKESIDE DR,GARLAND,TX,75042,57.0,9722720774,451120.0,Y,20121022.0,20120120
1,10100798122,1,"FIASCHETTI ENTERPRISES, LP",5610 5TH ST,KATY,TX,77493.0,101,8.329718e+09,PL,...,1830 SOUTH MASON RD STE 140,KATY,TX,77450,101.0,NaN,722511.0,Y,20050825.0,20051101
2,10103759501,1,"COLE HAAN COMPANY STORE, LLC",150 OCEAN RD C/O TAX DEPT.,GREENLAND,NH,3840.0,0,2.078463e+09,CI,...,HIGHLAND PARK VILLAGE STE 16,HIGHLAND PARK,TX,75205,57.0,NaN,448210.0,Y,19931108.0,19931114
3,10103759501,2,"COLE HAAN COMPANY STORE, LLC",150 OCEAN RD C/O TAX DEPT.,GREENLAND,NH,3840.0,0,2.078463e+09,CI,...,HIGHLAND VILLAGE 4005 WESTHEIMER,HOUSTON,TX,77027,101.0,2078462500,448210.0,Y,19941110.0,19941118
4,10103759501,3,"COLE HAAN COMPANY STORE, LLC",150 OCEAN RD C/O TAX DEPT.,GREENLAND,NH,3840.0,0,2.078463e+09,CI,...,3939 HWY 35 SOUTH STE 301,SAN MARCOS,TX,78666,105.0,5123960404,448210.0,Y,19971212.0,19980120


#### Taxpayer_Organizational_Name Column

In [12]:
# Taxpayer Organization Type:
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'AB'),'Taxpayer_Organizational_Name']='TEXAS BUSINESS ASSOC' # AB          TEXAS BUSINESS ASSOC            
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'AC'),'Taxpayer_Organizational_Name']='FRGN BUSINESS ASSOC' # AC          FRGN BUSINESS ASSOC             
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'AF'),'Taxpayer_Organizational_Name']='FOREIGN PROFESSIONAL ASSOCIATION' # AF          FOREIGN PROFESSIONAL ASSOCIATION
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'AP'),'Taxpayer_Organizational_Name']='TEXAS PROFESSIONAL ASSOCIATION' # AP          TEXAS PROFESSIONAL ASSOCIATION  
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'AR'),'Taxpayer_Organizational_Name']='OTHER ASSOCIATION' # AR          OTHER ASSOCIATION               
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'C'),'Taxpayer_Organizational_Name']='CORPORATION' # C           CORPORATION                     
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'CF'),'Taxpayer_Organizational_Name']='FOREIGN PROFIT CORPORATION' # CF          FOREIGN PROFIT CORPORATION      
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'CI'),'Taxpayer_Organizational_Name']='FOREIGN LMTD LIAB CO - OOS' # CI          FOREIGN LMTD LIAB CO - OOS      
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'CL'),'Taxpayer_Organizational_Name']='TEXAS LIMITED LIABILITY COMPANY' # CL          TEXAS LIMITED LIABILITY COMPANY 
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'CM'),'Taxpayer_Organizational_Name']='FOREIGN NON-PROFIT CORP - OOS' # CM          FOREIGN NON-PROFIT CORP - OOS   
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'CN'),'Taxpayer_Organizational_Name']='TEXAS NON-PROFIT CORPORATION' # CN          TEXAS NON-PROFIT CORPORATION    
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'CP'),'Taxpayer_Organizational_Name']='TEXAS PROFESSIONAL CORPORATION' # CP          TEXAS PROFESSIONAL CORPORATION
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'CR'),'Taxpayer_Organizational_Name']='TEXAS INSURANCE CORPORATION' # CR          TEXAS INSURANCE CORPORATION     
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'CS'),'Taxpayer_Organizational_Name']='FOREIGN INSURANCE CORP - OOS' # CS          FOREIGN INSURANCE CORP - OOS    
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'CT'),'Taxpayer_Organizational_Name']='TEXAS PROFIT CORPORATION' # CT          TEXAS PROFIT CORPORATION        
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'CU'),'Taxpayer_Organizational_Name']='FOREIGN PROFESSIONAL CORPORATION' # CU          FOREIGN PROFESSIONAL CORPORATION
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'CW'),'Taxpayer_Organizational_Name']='TEXAS RAILROAD CORPORATION' # CW          TEXAS RAILROAD CORPORATION                          
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'CX'),'Taxpayer_Organizational_Name']='FOREIGN RAILROAD CORPORATION' # CX          FOREIGN RAILROAD CORPORATION                        
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'ES'),'Taxpayer_Organizational_Name']='ESTATE' # ES          ESTATE                                              
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'FA'),'Taxpayer_Organizational_Name']='FINANCIAL INSTITUTION - STATE SAVINGS & LOAN - OOS' # FA          FINANCIAL INSTITUTION - STATE SAVINGS & LOAN - OOS  
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'FB'),'Taxpayer_Organizational_Name']='FINANCIAL INSTITUTION - STATE SAVINGS B ANK - TX' # FB          FINANCIAL INSTITUTION - STATE SAVINGS B ANK - TX    
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'FC'),'Taxpayer_Organizational_Name']='FINANCIAL INSTITUTION - FEDERAL CREDIT UNION' # FC          FINANCIAL INSTITUTION - FEDERAL CREDIT UNION        
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'FD'),'Taxpayer_Organizational_Name']='FINANCIAL INSTITUTION - STATE SAVINGS & LOAN - TX' # FD          FINANCIAL INSTITUTION - STATE SAVINGS & LOAN - TX   
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'FE'),'Taxpayer_Organizational_Name']='FINANCIAL INSTITUTION - FEDERAL SAVINGS & LOAN-TX' # FE          FINANCIAL INSTITUTION - FEDERAL SAVINGS & LOAN-TX   
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'FF'),'Taxpayer_Organizational_Name']='FINANCIAL INSTITUTION - FEDERAL BANK - TX' # FF          FINANCIAL INSTITUTION - FEDERAL BANK - TX           
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'FG'),'Taxpayer_Organizational_Name']='FINANCIAL INSTITUTION - FEDERAL SAVINGS BANK - TX' # FG          FINANCIAL INSTITUTION - FEDERAL SAVINGS BANK - TX   
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'FH'),'Taxpayer_Organizational_Name']='FINANCIAL INSTITUTION - STATE SAVINGS BANK - OOS' # FH          FINANCIAL INSTITUTION - STATE SAVINGS BANK - OOS    
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'FI'),'Taxpayer_Organizational_Name']='FINANCIAL INSTITUTION - STATE CREDIT UNION - TX' # FI          FINANCIAL INSTITUTION - STATE CREDIT UNION - TX     
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'FJ'),'Taxpayer_Organizational_Name']='FINANCIAL INSTITUTION - FEDERAL BANK - OOS' # FJ          FINANCIAL INSTITUTION - FEDERAL BANK - OOS          
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'FK'),'Taxpayer_Organizational_Name']='FINANCIAL INSTITUTION - FEDERAL SAVINGS BANK - OOS' # FK          FINANCIAL INSTITUTION - FEDERAL SAVINGS BANK - OOS  
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'FL'),'Taxpayer_Organizational_Name']='FINANCIAL INSTITUTION - STATE LIMITED BANK ASSOC' # FL          FINANCIAL INSTITUTION - STATE LIMITED BANK ASSOC    
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'FM'),'Taxpayer_Organizational_Name']='FINANCIAL INSTITUTION - TRUST COMPANY' # FM          FINANCIAL INSTITUTION - TRUST COMPANY              
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'FN'),'Taxpayer_Organizational_Name']='FINANCIAL INSTITUTION - FEDERAL SAVINGS & LOAN-OOS' # FN          FINANCIAL INSTITUTION - FEDERAL SAVINGS & LOAN-OOS 
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'FO'),'Taxpayer_Organizational_Name']='FINANCIAL INSTITUTION - STATE BANK - OOS' # FO          FINANCIAL INSTITUTION - STATE BANK - OOS           
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'FP'),'Taxpayer_Organizational_Name']='FINANCIAL INSTITUTION' # FP          FINANCIAL INSTITUTION                              
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'FR'),'Taxpayer_Organizational_Name']='FINANCIAL INSTITUTION - FOREIGN COUNTRY BANK' # FR          FINANCIAL INSTITUTION - FOREIGN COUNTRY BANK       
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'FS'),'Taxpayer_Organizational_Name']='FINANCIAL INSTITUTION - STATE BANK - TX' # FS          FINANCIAL INSTITUTION - STATE BANK - TX            
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'FT'),'Taxpayer_Organizational_Name']='FINANCIAL INSTITUTION - STATE CREDIT UNION - OOS' # FT          FINANCIAL INSTITUTION - STATE CREDIT UNION - OOS   
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'GC'),'Taxpayer_Organizational_Name']='CITY' # GC          CITY                                               
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'GD'),'Taxpayer_Organizational_Name']='FEDERAL AGENCY' # GD          FEDERAL AGENCY                                     
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'GF'),'Taxpayer_Organizational_Name']='STATE AGENCY - OOS' # GF          STATE AGENCY - OOS                                 
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'GJ'),'Taxpayer_Organizational_Name']='JUNIOR COLLEGE' # GJ          JUNIOR COLLEGE                                     
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'GL'),'Taxpayer_Organizational_Name']='LOCAL OFFICIAL' # GL          LOCAL OFFICIAL                                     
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'GM'),'Taxpayer_Organizational_Name']='MASS TRANSIT' # GM          MASS TRANSIT                                       
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'GO'),'Taxpayer_Organizational_Name']='COUNTY' # GO          COUNTY                                             
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'GP'),'Taxpayer_Organizational_Name']='SPECIAL PURPOSE DISTRICT' # GP          SPECIAL PURPOSE DISTRICT                           
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'GR'),'Taxpayer_Organizational_Name']='RAPID TRANSIT' # GR          RAPID TRANSIT              
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'GS'),'Taxpayer_Organizational_Name']='SCHOOL DISTRICT' # GS          SCHOOL DISTRICT            
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'GT'),'Taxpayer_Organizational_Name']='STATE AGENCY - TX' # GT          STATE AGENCY - TX          
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'GU'),'Taxpayer_Organizational_Name']='STATE COLLEGE/UNIVERSITY' # GU          STATE COLLEGE/UNIVERSITY   
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'GY'),'Taxpayer_Organizational_Name']='COMMUNITY COLLEGE' # GY          COMMUNITY COLLEGE          
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'HF'),'Taxpayer_Organizational_Name']='FRGN HOLDING COMPANY' # HF          FRGN HOLDING COMPANY       
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'IS'),'Taxpayer_Organizational_Name']='INDIVIDUAL - SOLE OWNER' # IS          INDIVIDUAL - SOLE OWNER    
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'J'),'Taxpayer_Organizational_Name']=' JOINT VENTURE' # J           JOINT VENTURE              
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'L'),'Taxpayer_Organizational_Name']=' LIMITED (LIABILITY) COMPANY' # L           LIMITED (LIABILITY) COMPANY
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'M'),'Taxpayer_Organizational_Name']=' LIMITED (LIABILITY) PARTNERSHIP' # M           LIMITED (LIABILITY) PARTNERSHIP 
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'O'),'Taxpayer_Organizational_Name']=' OTHER' # O           OTHER                           
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'P'),'Taxpayer_Organizational_Name']=' GENERAL PARTNERSHIP' # P           GENERAL PARTNERSHIP             
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'PB'),'Taxpayer_Organizational_Name']='BUS GENERAL PRTNSHP' # PB          BUS GENERAL PRTNSHP             
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'PF'),'Taxpayer_Organizational_Name']='FRGN LIMITED PRTNSHP' # PF          FRGN LIMITED PRTNSHP            
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'PI'),'Taxpayer_Organizational_Name']='IND GENERAL PRTNSHP' # PI          IND GENERAL PRTNSHP             
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'PL'),'Taxpayer_Organizational_Name']='TX LIMITED PRTNSHP' # PL          TX LIMITED PRTNSHP              
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'PO'),'Taxpayer_Organizational_Name']='OIL & GAS SPECIAL' # PO          OIL & GAS SPECIAL               
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'PV'),'Taxpayer_Organizational_Name']='TEXAS JOINT VENTURE' # PV          TEXAS JOINT VENTURE             
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'PW'),'Taxpayer_Organizational_Name']='FRGN JOINT VENTURE' # PW          FRGN JOINT VENTURE              
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'PX'),'Taxpayer_Organizational_Name']='TX LLP REGISTRATION' # PX          TX LLP REGISTRATION             
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'PY'),'Taxpayer_Organizational_Name']='FRGN LLP REGISTRATION' # PY          FRGN LLP REGISTRATION           
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'PZ'),'Taxpayer_Organizational_Name']='IND SUCCESSOR PRTSHP' # PZ          IND SUCCESSOR PRTSHP            
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'S'),'Taxpayer_Organizational_Name']=' SOLE PROPRIETORSHIP' # S           SOLE PROPRIETORSHIP             
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'SF'),'Taxpayer_Organizational_Name']='FRGN JOINT STOCK CO' # SF          FRGN JOINT STOCK CO             
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'ST'),'Taxpayer_Organizational_Name']='TEXAS JOINT STOCK CO' # ST          TEXAS JOINT STOCK CO            
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'TF'),'Taxpayer_Organizational_Name']='FOREIGN BUSINESS TRUST' # TF          FOREIGN BUSINESS TRUST               
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'TH'),'Taxpayer_Organizational_Name']='TX RL EST INV TRST' # TH          TX RL EST INV TRST                   
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'TI'),'Taxpayer_Organizational_Name']='FOREIGN REAL ESTATE INVESTMENT TRUST' # TI          FOREIGN REAL ESTATE INVESTMENT TRUST 
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'TR'),'Taxpayer_Organizational_Name']='TRUST' # TR          TRUST                                
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'UF'),'Taxpayer_Organizational_Name']='UNKNOWN - FRANCHISE' # UF          UNKNOWN - FRANCHISE                  
df_stact.loc[(df_stact.Taxpayer_Organizational_Type == 'UK'),'Taxpayer_Organizational_Name']='UNKNOWN' # UK          UNKNOWN                               
df_stact.head()

,Taxpayer_Number,Outlet_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Phone_Number,Taxpayer_Organizational_Type,...,Outlet_City,Outlet_State,Outlet_Zip_Code,Outlet_County_Code,Outlet_Phone_Number,Outlet_NAICS_Code,Outlet_Inside_Outside_City_Limits_Indicator,Outlet_Permit_Issue_Date,Outlet_First_Sales_Date,Taxpayer_Organizational_Name
0,10065041799,1,JOHNNY MARSHALL AND PAUL LIBBY,1616 LAKESIDE DR,GARLAND,TX,75042.0,57,9.722721e+09,PI,...,GARLAND,TX,75042,57.0,9722720774,451120.0,Y,20121022.0,20120120,IND GENERAL PRTNSHP
1,10100798122,1,"FIASCHETTI ENTERPRISES, LP",5610 5TH ST,KATY,TX,77493.0,101,8.329718e+09,PL,...,KATY,TX,77450,101.0,NaN,722511.0,Y,20050825.0,20051101,TX LIMITED PRTNSHP
2,10103759501,1,"COLE HAAN COMPANY STORE, LLC",150 OCEAN RD C/O TAX DEPT.,GREENLAND,NH,3840.0,0,2.078463e+09,CI,...,HIGHLAND PARK,TX,75205,57.0,NaN,448210.0,Y,19931108.0,19931114,FOREIGN LMTD LIAB CO - OOS
3,10103759501,2,"COLE HAAN COMPANY STORE, LLC",150 OCEAN RD C/O TAX DEPT.,GREENLAND,NH,3840.0,0,2.078463e+09,CI,...,HOUSTON,TX,77027,101.0,2078462500,448210.0,Y,19941110.0,19941118,FOREIGN LMTD LIAB CO - OOS
4,10103759501,3,"COLE HAAN COMPANY STORE, LLC",150 OCEAN RD C/O TAX DEPT.,GREENLAND,NH,3840.0,0,2.078463e+09,CI,...,SAN MARCOS,TX,78666,105.0,5123960404,448210.0,Y,19971212.0,19980120,FOREIGN LMTD LIAB CO - OOS


### Formating date
* Replacing NaN to 0, converting float to int, filtering 0, and formating date

In [13]:
df_stact['Outlet_Permit_Issue_Date'] = df_stact['Outlet_Permit_Issue_Date'].fillna(0)
df_stact['Outlet_First_Sales_Date'] = df_stact['Outlet_First_Sales_Date'].fillna(0)
df_stact['Taxpayer_Zip_Code'] = df_stact['Taxpayer_Zip_Code'].fillna(0)

df_stact['Outlet_Permit_Issue_Date'] = df_stact['Outlet_Permit_Issue_Date'].astype(np.int64)
df_stact['Taxpayer_Zip_Code'] = df_stact['Taxpayer_Zip_Code'].astype(np.int64)

df_stact = df_stact[df_stact['Outlet_Permit_Issue_Date']!=0]
df_stact = df_stact[df_stact['Outlet_First_Sales_Date']!=0]

df_stact.head(3)

,Taxpayer_Number,Outlet_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Phone_Number,Taxpayer_Organizational_Type,...,Outlet_City,Outlet_State,Outlet_Zip_Code,Outlet_County_Code,Outlet_Phone_Number,Outlet_NAICS_Code,Outlet_Inside_Outside_City_Limits_Indicator,Outlet_Permit_Issue_Date,Outlet_First_Sales_Date,Taxpayer_Organizational_Name
0,10065041799,1,JOHNNY MARSHALL AND PAUL LIBBY,1616 LAKESIDE DR,GARLAND,TX,75042,57,9.722721e+09,PI,...,GARLAND,TX,75042,57.0,9722720774,451120.0,Y,20121022,20120120,IND GENERAL PRTNSHP
1,10100798122,1,"FIASCHETTI ENTERPRISES, LP",5610 5TH ST,KATY,TX,77493,101,8.329718e+09,PL,...,KATY,TX,77450,101.0,NaN,722511.0,Y,20050825,20051101,TX LIMITED PRTNSHP
2,10103759501,1,"COLE HAAN COMPANY STORE, LLC",150 OCEAN RD C/O TAX DEPT.,GREENLAND,NH,3840,0,2.078463e+09,CI,...,HIGHLAND PARK,TX,75205,57.0,NaN,448210.0,Y,19931108,19931114,FOREIGN LMTD LIAB CO - OOS


#### FTACT DF created

In [14]:
df_ftact.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,...,Responsibility_End_Date,Responsibility_End_Reason_Code,Secretary_of_State_File_Number,SOS_Charter_Date,SOS_Status_Date,SOS_Status_Code,Rigth_to_Tansact_Business_Code,Current_Exempt_Reason_Code,Exempt_Begin_Date,NAICS_Code
0,10004813456,MARSHALL E HAMILTON MD PA.,PO BOX 38206 C/O STEVE WINN CPA,DALLAS,TX,75238.0,57,AP,9726442142,X,...,NaN,0,NaN,NaN,NaN,NaN,A,NaN,NaN,621111.0
1,10058125732,CAP INVESTMENT GROUP,1801 HINKLE DR STE 150,DENTON,TX,76201.0,61,PB,9408914223,X,...,NaN,0,NaN,NaN,NaN,NaN,A,NaN,NaN,551112.0
2,10082832105,"JDM RESOURCES, LTD",5105 BUCKLAND DR,MCKINNEY,TX,75070.0,43,PL,2145440124,X,...,NaN,0,NaN,NaN,NaN,NaN,A,NaN,NaN,525990.0
3,10100464006,CLARK INSURANCE,1945 CONGRESS ST STE A,PORTLAND,ME,4102.0,0,CF,2075232218,V,...,NaN,0,800878850.0,20071001.0,20071001.0,A,A,NaN,NaN,NaN
4,10100798122,"FIASCHETTI ENTERPRISES, LP",5610 5TH ST,KATY,TX,77493.0,101,PL,8329717661,U,...,NaN,0,800242075.0,20030826.0,20090520.0,R,A,NaN,NaN,NaN


#### Taxpayer_Organizational_Name and Record_Type_Name Column

In [15]:
# Taxpayer Organization Type:
#   AB – Texas Business Association
#   AC – Foreign Business Association
#   AF – Foreign Professional Association
#   AP – Texas Professional Association
#   AR – Other Association
#   CF - Foreign Profit
#   CI - Limited Liability Company - Foreign
#   CL - Limited Liability Company - Texas
#   CM - Foreign Non-Profit
#   CN - Texas Non-Profit
#   CP - Professional
#   CR - Texas Insurance
#   CS - Foreign Insurance - OOS
#   CT - Texas Profit
#   CU – Foreign Professional Corporation
#   CW - Texas Railroad Corporation
#   CX - Foreign Railroad Corporation – OOS
#   HF – Foreign Holding Company
#   PB – Business General Partnership
#   PF – Foreign Limited Partnership
#   PI – Individual General Partnership
#   PL – Texas Limited Partnership
#   PV – Texas Joint Venture
#   PW – Foreign Joint Venture
#   PX – Texas Limited Liability Partnership
#   PY – Foreign Limited Liability Partnerhsip
#   SF – Foreign Joint Stock Company
#   ST – Texas Joint Stock Company
#   TF – Foreign Business Trust
#   TH – Texas Real Estate Investment Trust
#   TI – Foreign Real Estate Investment Trust
#   TR – Texas Business Trust
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'AB'),'Taxpayer_Organizational_Name']='Texas Business Association'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'AC'),'Taxpayer_Organizational_Name']='Foreign Business Association'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'AF'),'Taxpayer_Organizational_Name']='Foreign Professional Association'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'AP'),'Taxpayer_Organizational_Name']='Texas Professional Association'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'AR'),'Taxpayer_Organizational_Name']='Other Association'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CF'),'Taxpayer_Organizational_Name']='Foreign Profit'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CI'),'Taxpayer_Organizational_Name']='Limited Liability Company - Foreign'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CL'),'Taxpayer_Organizational_Name']='Limited Liability Company - Texas'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CM'),'Taxpayer_Organizational_Name']='Foreign Non-Profit'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CN'),'Taxpayer_Organizational_Name']='Texas Non-Profit'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CP'),'Taxpayer_Organizational_Name']='Professional'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CR'),'Taxpayer_Organizational_Name']='Texas Insurance'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CS'),'Taxpayer_Organizational_Name']='Foreign Insurance - OOS'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CT'),'Taxpayer_Organizational_Name']='Texas Profit'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CU'),'Taxpayer_Organizational_Name']='Foreign Professional Corporation'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CW'),'Taxpayer_Organizational_Name']='Texas Railroad Corporation'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CX'),'Taxpayer_Organizational_Name']='Foreign Railroad Corporation - OOS'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'HF'),'Taxpayer_Organizational_Name']='Foreign Holding Company'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'PB'),'Taxpayer_Organizational_Name']='Business General Partnership'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'PF'),'Taxpayer_Organizational_Name']='Foreign Limited Partnership'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'PI'),'Taxpayer_Organizational_Name']='Individual General Partnership'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'PL'),'Taxpayer_Organizational_Name']='Texas Limited Partnership'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'PV'),'Taxpayer_Organizational_Name']='Texas Joint Venture'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'PW'),'Taxpayer_Organizational_Name']='Foreign Joint Venture'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'PX'),'Taxpayer_Organizational_Name']='Texas Limited Liability Partnership'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'PY'),'Taxpayer_Organizational_Name']='Foreign Limited Liability Partnerhsip'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'SF'),'Taxpayer_Organizational_Name']='Foreign Joint Stock Company'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'ST'),'Taxpayer_Organizational_Name']='Texas Joint Stock Company'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'TF'),'Taxpayer_Organizational_Name']='Foreign Business Trust'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'TH'),'Taxpayer_Organizational_Name']='Texas Real Estate Investment Trust'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'TI'),'Taxpayer_Organizational_Name']='Foreign Real Estate Investment Trust'
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'TR'),'Taxpayer_Organizational_Name']='Texas Business Trust'

# Record Type Code:
#   U = Secretary of State (SOS) File Number
#   V = SOS Certificate of Authority (COA) File Number
#   X = Comptroller Assigned File Number

# SOS Charter/COA:
#   Depending on the Record Type Code value, this number
#   is the SOS, COA or Comptroller Assigned File Number.
#   If the Record Type Code is an 'X', this field will be
#   blank.  They do not have a current SOS Charter/COA.
df_ftact.loc[(df_ftact.Record_Type_Code == 'U'),'Record_Type_Name']='Secretary of State (SOS) File Number'
df_ftact.loc[(df_ftact.Record_Type_Code == 'V'),'Record_Type_Name']='SOS Certificate of Authority (COA) File Number'
df_ftact.loc[(df_ftact.Record_Type_Code == 'X'),'Record_Type_Name']='Comptroller Assigned File Number'
df_ftact.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,...,Secretary_of_State_File_Number,SOS_Charter_Date,SOS_Status_Date,SOS_Status_Code,Rigth_to_Tansact_Business_Code,Current_Exempt_Reason_Code,Exempt_Begin_Date,NAICS_Code,Taxpayer_Organizational_Name,Record_Type_Name
0,10004813456,MARSHALL E HAMILTON MD PA.,PO BOX 38206 C/O STEVE WINN CPA,DALLAS,TX,75238.0,57,AP,9726442142,X,...,NaN,NaN,NaN,NaN,A,NaN,NaN,621111.0,Texas Professional Association,Comptroller Assigned File Number
1,10058125732,CAP INVESTMENT GROUP,1801 HINKLE DR STE 150,DENTON,TX,76201.0,61,PB,9408914223,X,...,NaN,NaN,NaN,NaN,A,NaN,NaN,551112.0,Business General Partnership,Comptroller Assigned File Number
2,10082832105,"JDM RESOURCES, LTD",5105 BUCKLAND DR,MCKINNEY,TX,75070.0,43,PL,2145440124,X,...,NaN,NaN,NaN,NaN,A,NaN,NaN,525990.0,Texas Limited Partnership,Comptroller Assigned File Number
3,10100464006,CLARK INSURANCE,1945 CONGRESS ST STE A,PORTLAND,ME,4102.0,0,CF,2075232218,V,...,800878850.0,20071001.0,20071001.0,A,A,NaN,NaN,NaN,Foreign Profit,SOS Certificate of Authority (COA) File Number
4,10100798122,"FIASCHETTI ENTERPRISES, LP",5610 5TH ST,KATY,TX,77493.0,101,PL,8329717661,U,...,800242075.0,20030826.0,20090520.0,R,A,NaN,NaN,NaN,Texas Limited Partnership,Secretary of State (SOS) File Number


#### Responsibility_End_Reason_Name column

In [16]:
# Responsibility End Reason Code:
#   This is for mostly for Record Type Code value 'X'.
#     00 = Active or Inactive with no Reason Code
#     01 = Discountinued Doing Business
#     02 = Dissolved in Home State
#     03 = Merged Out of Existence
#     04 = Converted
#     05 = Consolidated
#     06 = Forfeited in Home State
#     08 = No Nexus
#     09 = No Nexus – Dates not the same
#     11 = Special Information Report
df_ftact.loc[(df_ftact.Responsibility_End_Reason_Code == 0),'Responsibility_End_Reason_Name']='Active or Inactive with no Reason Code'
df_ftact.loc[(df_ftact.Responsibility_End_Reason_Code == 1),'Responsibility_End_Reason_Name']='Discountinued Doing Business'
df_ftact.loc[(df_ftact.Responsibility_End_Reason_Code == 2),'Responsibility_End_Reason_Name']='Dissolved in Home State'
df_ftact.loc[(df_ftact.Responsibility_End_Reason_Code == 3),'Responsibility_End_Reason_Name']='Merged Out of Existence'
df_ftact.loc[(df_ftact.Responsibility_End_Reason_Code == 4),'Responsibility_End_Reason_Name']='Converted'
df_ftact.loc[(df_ftact.Responsibility_End_Reason_Code == 5),'Responsibility_End_Reason_Name']='Consolidated'
df_ftact.loc[(df_ftact.Responsibility_End_Reason_Code == 6),'Responsibility_End_Reason_Name']='Forfeited in Home State'
df_ftact.loc[(df_ftact.Responsibility_End_Reason_Code == 8),'Responsibility_End_Reason_Name']='No Nexus'
df_ftact.loc[(df_ftact.Responsibility_End_Reason_Code == 9),'Responsibility_End_Reason_Name']='No Nexus – Dates not the same'
df_ftact.loc[(df_ftact.Responsibility_End_Reason_Code == 11),'Responsibility_End_Reason_Name']='Special Information Report'
df_ftact.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,...,SOS_Charter_Date,SOS_Status_Date,SOS_Status_Code,Rigth_to_Tansact_Business_Code,Current_Exempt_Reason_Code,Exempt_Begin_Date,NAICS_Code,Taxpayer_Organizational_Name,Record_Type_Name,Responsibility_End_Reason_Name
0,10004813456,MARSHALL E HAMILTON MD PA.,PO BOX 38206 C/O STEVE WINN CPA,DALLAS,TX,75238.0,57,AP,9726442142,X,...,NaN,NaN,NaN,A,NaN,NaN,621111.0,Texas Professional Association,Comptroller Assigned File Number,Active or Inactive with no Reason Code
1,10058125732,CAP INVESTMENT GROUP,1801 HINKLE DR STE 150,DENTON,TX,76201.0,61,PB,9408914223,X,...,NaN,NaN,NaN,A,NaN,NaN,551112.0,Business General Partnership,Comptroller Assigned File Number,Active or Inactive with no Reason Code
2,10082832105,"JDM RESOURCES, LTD",5105 BUCKLAND DR,MCKINNEY,TX,75070.0,43,PL,2145440124,X,...,NaN,NaN,NaN,A,NaN,NaN,525990.0,Texas Limited Partnership,Comptroller Assigned File Number,Active or Inactive with no Reason Code
3,10100464006,CLARK INSURANCE,1945 CONGRESS ST STE A,PORTLAND,ME,4102.0,0,CF,2075232218,V,...,20071001.0,20071001.0,A,A,NaN,NaN,NaN,Foreign Profit,SOS Certificate of Authority (COA) File Number,Active or Inactive with no Reason Code
4,10100798122,"FIASCHETTI ENTERPRISES, LP",5610 5TH ST,KATY,TX,77493.0,101,PL,8329717661,U,...,20030826.0,20090520.0,R,A,NaN,NaN,NaN,Texas Limited Partnership,Secretary of State (SOS) File Number,Active or Inactive with no Reason Code


#### SOS_Status_Name Column

In [17]:
# SOS Charter/COA:
#   Depending on the Record Type Code value, this number
#   is the SOS, COA or Comptroller Assigned File Number.
#   If the Record Type Code is an 'X', this field will be
#   blank.  They do not have a current SOS Charter/COA.

# SOS Status Code:
#   For Charter/COA Numbers:
#     A = Active
#     B = Consolidated
#     C = Converted
#     D = Dissolved
#     E = Expired
#     F = Forfeited Franchise Tax
#     G = Miscellaneous
#     I = Closed by FDIC
#     J = State Charter Pulled
#     K = Forfeited Registered Agent
#     L = Forfeited Registered Office
#     M = Merger
#     N = Forfeited Hot Check
#     P = Forfeited Court Order
#     R = Reinstated
#     T = Terminated
#     W = Withdrawn
#     Y = Dead at Conversion 69
#     Z = Dead at Conversion 83
df_ftact.loc[(df_ftact.SOS_Status_Code == 'A'),'SOS_Status_Name']='Active'
df_ftact.loc[(df_ftact.SOS_Status_Code == 'B'),'SOS_Status_Name']='Consolidated'
df_ftact.loc[(df_ftact.SOS_Status_Code == 'C'),'SOS_Status_Name']='Converted'
df_ftact.loc[(df_ftact.SOS_Status_Code == 'D'),'SOS_Status_Name']='Dissolved'
df_ftact.loc[(df_ftact.SOS_Status_Code == 'E'),'SOS_Status_Name']='Expired'
df_ftact.loc[(df_ftact.SOS_Status_Code == 'F'),'SOS_Status_Name']='Forfeited Franchise Tax'
df_ftact.loc[(df_ftact.SOS_Status_Code == 'G'),'SOS_Status_Name']='Miscellaneous'
df_ftact.loc[(df_ftact.SOS_Status_Code == 'I'),'SOS_Status_Name']='Closed by FDIC'
df_ftact.loc[(df_ftact.SOS_Status_Code == 'J'),'SOS_Status_Name']='State Charter Pulled'
df_ftact.loc[(df_ftact.SOS_Status_Code == 'K'),'SOS_Status_Name']='Forfeited Registered Agent'
df_ftact.loc[(df_ftact.SOS_Status_Code == 'L'),'SOS_Status_Name']='Forfeited Registered Office'
df_ftact.loc[(df_ftact.SOS_Status_Code == 'M'),'SOS_Status_Name']='Merger'
df_ftact.loc[(df_ftact.SOS_Status_Code == 'N'),'SOS_Status_Name']='Forfeited Hot Check'
df_ftact.loc[(df_ftact.SOS_Status_Code == 'P'),'SOS_Status_Name']='Forfeited Court Order'
df_ftact.loc[(df_ftact.SOS_Status_Code == 'R'),'SOS_Status_Name']='Reinstated'
df_ftact.loc[(df_ftact.SOS_Status_Code == 'T'),'SOS_Status_Name']='Terminated'
df_ftact.loc[(df_ftact.SOS_Status_Code == 'W'),'SOS_Status_Name']='Withdrawn'
df_ftact.loc[(df_ftact.SOS_Status_Code == 'Y'),'SOS_Status_Name']='Dead at Conversion 69'
df_ftact.loc[(df_ftact.SOS_Status_Code == 'Z'),'SOS_Status_Name']='Dead at Conversion 83'
df_ftact.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,...,SOS_Status_Date,SOS_Status_Code,Rigth_to_Tansact_Business_Code,Current_Exempt_Reason_Code,Exempt_Begin_Date,NAICS_Code,Taxpayer_Organizational_Name,Record_Type_Name,Responsibility_End_Reason_Name,SOS_Status_Name
0,10004813456,MARSHALL E HAMILTON MD PA.,PO BOX 38206 C/O STEVE WINN CPA,DALLAS,TX,75238.0,57,AP,9726442142,X,...,NaN,NaN,A,NaN,NaN,621111.0,Texas Professional Association,Comptroller Assigned File Number,Active or Inactive with no Reason Code,NaN
1,10058125732,CAP INVESTMENT GROUP,1801 HINKLE DR STE 150,DENTON,TX,76201.0,61,PB,9408914223,X,...,NaN,NaN,A,NaN,NaN,551112.0,Business General Partnership,Comptroller Assigned File Number,Active or Inactive with no Reason Code,NaN
2,10082832105,"JDM RESOURCES, LTD",5105 BUCKLAND DR,MCKINNEY,TX,75070.0,43,PL,2145440124,X,...,NaN,NaN,A,NaN,NaN,525990.0,Texas Limited Partnership,Comptroller Assigned File Number,Active or Inactive with no Reason Code,NaN
3,10100464006,CLARK INSURANCE,1945 CONGRESS ST STE A,PORTLAND,ME,4102.0,0,CF,2075232218,V,...,20071001.0,A,A,NaN,NaN,NaN,Foreign Profit,SOS Certificate of Authority (COA) File Number,Active or Inactive with no Reason Code,Active
4,10100798122,"FIASCHETTI ENTERPRISES, LP",5610 5TH ST,KATY,TX,77493.0,101,PL,8329717661,U,...,20090520.0,R,A,NaN,NaN,NaN,Texas Limited Partnership,Secretary of State (SOS) File Number,Active or Inactive with no Reason Code,Reinstated


#### Rigth_to_Tansact_Business_Name Column

In [18]:
# Right to Transact Business Code:
#   A = Active
#   D = Active – Eligible for Termination/Withdrawl
#   N = Forfeited
#   I = Franchise Tax Involuntarily Ended
#   U = Franchise Tax Not Established
#   blank = Franchise Tax Ended

# Exempt Reason Code:
#   blank = Not Exempt
#   rest = Exempt for various reasons.  A list of value descriptions
#          may be requested separately.
df_ftact.loc[(df_ftact.Rigth_to_Tansact_Business_Code == 'A'),'Rigth_to_Tansact_Business_Name']='Active'
df_ftact.loc[(df_ftact.Rigth_to_Tansact_Business_Code == 'D'),'Rigth_to_Tansact_Business_Name']='Active – Eligible for Termination/Withdrawl'
df_ftact.loc[(df_ftact.Rigth_to_Tansact_Business_Code == 'N'),'Rigth_to_Tansact_Business_Name']='Forfeited'
df_ftact.loc[(df_ftact.Rigth_to_Tansact_Business_Code == 'I'),'Rigth_to_Tansact_Business_Name']='Franchise Tax Involuntarily Ended'
df_ftact.loc[(df_ftact.Rigth_to_Tansact_Business_Code == 'U'),'Rigth_to_Tansact_Business_Name']='Franchise Tax Not Established'
df_ftact.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,...,SOS_Status_Code,Rigth_to_Tansact_Business_Code,Current_Exempt_Reason_Code,Exempt_Begin_Date,NAICS_Code,Taxpayer_Organizational_Name,Record_Type_Name,Responsibility_End_Reason_Name,SOS_Status_Name,Rigth_to_Tansact_Business_Name
0,10004813456,MARSHALL E HAMILTON MD PA.,PO BOX 38206 C/O STEVE WINN CPA,DALLAS,TX,75238.0,57,AP,9726442142,X,...,NaN,A,NaN,NaN,621111.0,Texas Professional Association,Comptroller Assigned File Number,Active or Inactive with no Reason Code,NaN,Active
1,10058125732,CAP INVESTMENT GROUP,1801 HINKLE DR STE 150,DENTON,TX,76201.0,61,PB,9408914223,X,...,NaN,A,NaN,NaN,551112.0,Business General Partnership,Comptroller Assigned File Number,Active or Inactive with no Reason Code,NaN,Active
2,10082832105,"JDM RESOURCES, LTD",5105 BUCKLAND DR,MCKINNEY,TX,75070.0,43,PL,2145440124,X,...,NaN,A,NaN,NaN,525990.0,Texas Limited Partnership,Comptroller Assigned File Number,Active or Inactive with no Reason Code,NaN,Active
3,10100464006,CLARK INSURANCE,1945 CONGRESS ST STE A,PORTLAND,ME,4102.0,0,CF,2075232218,V,...,A,A,NaN,NaN,NaN,Foreign Profit,SOS Certificate of Authority (COA) File Number,Active or Inactive with no Reason Code,Active,Active
4,10100798122,"FIASCHETTI ENTERPRISES, LP",5610 5TH ST,KATY,TX,77493.0,101,PL,8329717661,U,...,R,A,NaN,NaN,NaN,Texas Limited Partnership,Secretary of State (SOS) File Number,Active or Inactive with no Reason Code,Reinstated,Active


#### Formating data
* changing float to int

In [19]:
df_ftact['Taxpayer_Zip_Code'] = df_ftact['Taxpayer_Zip_Code'].fillna(0)
df_ftact['Taxpayer_Zip_Code'] = df_ftact['Taxpayer_Zip_Code'].astype(np.int64)
df_ftact = df_ftact[df_ftact['Taxpayer_Zip_Code']!=0]
df_ftact.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,...,SOS_Status_Code,Rigth_to_Tansact_Business_Code,Current_Exempt_Reason_Code,Exempt_Begin_Date,NAICS_Code,Taxpayer_Organizational_Name,Record_Type_Name,Responsibility_End_Reason_Name,SOS_Status_Name,Rigth_to_Tansact_Business_Name
0,10004813456,MARSHALL E HAMILTON MD PA.,PO BOX 38206 C/O STEVE WINN CPA,DALLAS,TX,75238,57,AP,9726442142,X,...,NaN,A,NaN,NaN,621111.0,Texas Professional Association,Comptroller Assigned File Number,Active or Inactive with no Reason Code,NaN,Active
1,10058125732,CAP INVESTMENT GROUP,1801 HINKLE DR STE 150,DENTON,TX,76201,61,PB,9408914223,X,...,NaN,A,NaN,NaN,551112.0,Business General Partnership,Comptroller Assigned File Number,Active or Inactive with no Reason Code,NaN,Active
2,10082832105,"JDM RESOURCES, LTD",5105 BUCKLAND DR,MCKINNEY,TX,75070,43,PL,2145440124,X,...,NaN,A,NaN,NaN,525990.0,Texas Limited Partnership,Comptroller Assigned File Number,Active or Inactive with no Reason Code,NaN,Active
3,10100464006,CLARK INSURANCE,1945 CONGRESS ST STE A,PORTLAND,ME,4102,0,CF,2075232218,V,...,A,A,NaN,NaN,NaN,Foreign Profit,SOS Certificate of Authority (COA) File Number,Active or Inactive with no Reason Code,Active,Active
4,10100798122,"FIASCHETTI ENTERPRISES, LP",5610 5TH ST,KATY,TX,77493,101,PL,8329717661,U,...,R,A,NaN,NaN,NaN,Texas Limited Partnership,Secretary of State (SOS) File Number,Active or Inactive with no Reason Code,Reinstated,Active


##### FTOFFDIR DF created

In [20]:
df_ftoffdir.head()

,Taxpayer_Number,Sequence_Number,Director_Latest_Year_Filed,Director_Name,Director_Title,Director_Address,Director_City,Director_State,Director_Zip_Code
0,10100464006,1,2019.0,GEORGE J SHAW,PRESIDENT,5 BRENDA CIRCLE,SACO,ME,04072
1,10100464006,2,2019.0,GEORGE J SHAW,DIRECTOR,5 BRENDA CIRCLE,SACO,ME,04072
2,10100464006,3,2019.0,JEFFREY A LIND,TREASURER,60 LINDSEY DRIVE,STANDISH,ME,04084
3,10100464006,4,2019.0,JEFFREY A LIND,DIRECTOR,60 LINDSEY DRIVE,STANDISH,ME,04084
4,10100464006,5,2019.0,GEORGE L RAMSDELL,SENIOR VIC,30 TALL PINES ROAD,SCARBOROUGH,ME,04074


#### Formating data
* changing float to int

In [21]:
df_ftoffdir['Director_Latest_Year_Filed'] = df_ftoffdir['Director_Latest_Year_Filed'].fillna(0)
df_ftoffdir['Director_Latest_Year_Filed'] = df_ftoffdir['Director_Latest_Year_Filed'].astype(np.int64)
df_ftoffdir.head()

,Taxpayer_Number,Sequence_Number,Director_Latest_Year_Filed,Director_Name,Director_Title,Director_Address,Director_City,Director_State,Director_Zip_Code
0,10100464006,1,2019,GEORGE J SHAW,PRESIDENT,5 BRENDA CIRCLE,SACO,ME,04072
1,10100464006,2,2019,GEORGE J SHAW,DIRECTOR,5 BRENDA CIRCLE,SACO,ME,04072
2,10100464006,3,2019,JEFFREY A LIND,TREASURER,60 LINDSEY DRIVE,STANDISH,ME,04084
3,10100464006,4,2019,JEFFREY A LIND,DIRECTOR,60 LINDSEY DRIVE,STANDISH,ME,04084
4,10100464006,5,2019,GEORGE L RAMSDELL,SENIOR VIC,30 TALL PINES ROAD,SCARBOROUGH,ME,04074


## Upload DF's to Database
* Adding database connection
* Defining the Engine
** I was getting charmap error when attempting to drop the data to the database. I defined encoding = utf-8, yet it still did not work. Only when I hardcoded charset within the engine string is when the error finally went away.

In [30]:
connection_string = f"{mysql_user}:{mysql_password}@localhost:3306/{db_name}?charset=utf8"
engine = create_engine(f'mysql://{connection_string}')

In [31]:
engine.table_names()

['ccim',
 'franchise_tax_info',
 'franchise_tax_info_date',
 'ftoffdir',
 'sales_tax_info',
 'sales_tax_info_date']

#### Creating two variables for today's date and today's datetime

In [32]:
currentDT = datetime.datetime.now()
DateTimeSent = currentDT.strftime("%Y-%m-%d %H:%M:%S")
dateCSV = currentDT.strftime("%Y-%m-%d")
print(dateCSV)
print(DateTimeSent)

2020-03-22
2020-03-22 23:04:57


## Calling Database tables for crosreferencing df data, to have non-duplicated data
* Grabing data from the database and storing the tax number column into a dataframe

In [13]:
ftact_in_db = pdsql.read_sql("SELECT Taxpayer_Number FROM franchise_tax_info",engine)
stact_in_db = pdsql.read_sql("SELECT Taxpayer_Number FROM sales_tax_info", engine)
ftoffdir_in_db = pdsql.read_sql("SELECT Taxpayer_Number FROM ftoffdir", engine)


print(f"Data count for ftact from the database : {len(ftact_in_db)}\nData count for stact from the Database: {len(stact_in_db)}\nData count for ftoffdir from the Database: {len(ftoffdir_in_db)}")

try:
    if df_fran.size != 0:
        print(f"\nData count from the new df data for df_fran: {len(df_fran)}")
except Exception as e:
    print("df_fran does not exist. Check your data source if it is available")
    
try:
    if df_stp.size != 0:
        print(f"Data count from the new df data for df_stp: {len(df_stp)}")
except Exception as e:
    print("df_stp does not exist. Check your data source if it is available")
    
try:
    if df_ftact.size != 0:
        print(f"Data count from the new df data for df_ftact: {len(df_ftact)}")
except Exception as e:
    print("df_ftact does not exist. Check your data source if it is available")
    
try:
    if df_stact.size != 0:
        print(f"Data count from the new df data for df_stact: {len(df_stact)}")
except Exception as e:
    print("df_stact does not exist. Check your data source if it is available")
    
try:
    if df_ftoffdir.size != 0:
        print(f"Data count from the new df data for df_ftoffdir: {len(df_ftoffdir)}")
except Exception as e:
    print("df_ftoffdir does not exist. Check your data source if it is available")
    

Data count for ftact from the database : 4236082
Data count for stact from the Database: 1552010
Data count for ftoffdir from the Database: 7532736

Data count from the new df data for df_fran: 131232

Data count from the new df data for df_stp: 32738
df_ftact does not exist. Check your data source if it is available
df_stact does not exist. Check your data source if it is available
df_ftoffdir does not exist. Check your data source if it is available


## FTACT aka df_ftact
#### Checking table df with df data to make sure their are not duplicate tax paying numbers
* filtering new ftact with data from the database
* Checking data for ftact and also adding a new column of today's date and time
* Appending new companies (df_ftact) to  csv and Database

In [45]:
try:
    df_ftact = df_ftact[~df_ftact['Taxpayer_Number'].astype(int).isin(ftact_in_db['Taxpayer_Number'].astype(int))]
    if df_ftact.size != 0:
        df_ftact['DateTime'] = DateTimeSent
        print(f"There are {len(ftact_in_db)} data attributes in ftact table from the database\n{len(df_ftact)} new companies, based on tax payer number from filtered data df_tact")
        formattedData = os.path.abspath('HotelOccupancyTaxData/formattedData/'+ 'ftact_' + dateCSV + '.csv')
        df_ftact.to_csv(formattedData, index = False)
        df_ftact.to_sql(name='franchise_tax_info', con=engine, if_exists='append', index=False, chunksize=1000)
        print(f"ftact to database append, completed")
        f = open('HotelOccupancyTaxData/formattedData/DBUploadRecord.txt','a+')
        f.write(f'{DateTimeSent}\nftact_{dateCSV}.csv, {len(df_ftact)}, franchise_tax_info table, {len(ftact_in_db)}\n')
        f.close()
    else:
        print("No new data")
        f = open('HotelOccupancyTaxData/formattedData/DBUploadRecord.txt','a+')
        f.write(f'{DateTimeSent}\nftact_{dateCSV}.csv, {len(df_ftact)}, franchise_tax_info table, {len(ftact_in_db)}\n')
        f.close()
except Exception as e:
    print(f"Something went wrong, df_ftact was not able to append to database or no new data: {e}")

No new data


## STACT aka df_stact
#### Checking table df with df data to make sure their are not duplicate tax paying numbers
* filtering new stact with data from the database
* Checking data for stact and also adding a new column of today's date and time
* Appending new companies (df_stact) to  csv and Database

In [46]:
try:
    df_stact = df_stact[~df_stact['Taxpayer_Number'].astype(int).isin(stact_in_db['Taxpayer_Number'].astype(int))]
    if df_stact.size != 0:
        df_stact['DateTime'] = DateTimeSent
        print(f"There are {len(stact_in_db)} data attributes in stact table from the database\n{len(df_stact)} new companies, based on tax payer number from filtered data sf_tact")
        formattedData = os.path.abspath('HotelOccupancyTaxData/formattedData/'+ 'stact_' + dateCSV + '.csv')
        df_stact.to_csv(formattedData, index = False)
        df_stact.to_sql(name='sales_tax_info', con=engine, if_exists='append', index=False, chunksize=1000)
        print(f"stact to database append, completed")
        f = open('HotelOccupancyTaxData/formattedData/DBUploadRecord.txt','a+')
        f.write(f'stact_{dateCSV}.csv, {len(df_stact)}, sales_tax_info table, {len(stact_in_db)}\n')
        f.close()
    else:
        print("No new data")
        f = open('HotelOccupancyTaxData/formattedData/DBUploadRecord.txt','a+')
        f.write(f'stact_{dateCSV}.csv, {len(df_stact)}, sales_tax_info table, {len(stact_in_db)}\n')
        f.close()
except Exception as e:
    print(f'No new data for STACT: {e}')    

No new data


## FTOFFDIR aka df_ftoffdir
#### Checking table df with df data to make sure their are not duplicate tax paying numbers
* filtering new ftoffdir with data from the database
* Checking data for ftoffdir and also adding a new column of today's date and time
* Appending new companies (ftoffdir) to  csv and Database

In [47]:
try:
    df_ftoffdir = df_ftoffdir[~df_ftoffdir['Taxpayer_Number'].astype(int).isin(ftoffdir_in_db['Taxpayer_Number'].astype(int))]
    if df_ftoffdir.size != 0:
        df_ftoffdir['DateTime'] = DateTimeSent
        print(f"There are {len(ftoffdir_in_db)} data attributes in ftoffdir table from the database\n{len(df_ftoffdir)} new companies, based on tax payer number from filtered data df_ftoffdir")
        formattedData = os.path.abspath('HotelOccupancyTaxData/formattedData/'+ 'df_ftoffdir_' + dateCSV + '.csv')
        df_ftoffdir.to_csv(formattedData, index = False)
        df_ftoffdir.to_sql(name='ftoffdir', con=engine, if_exists='append', index=False, chunksize=1000)
        print(f"ftoffdir to database append, completed")
        f = open('HotelOccupancyTaxData/formattedData/DBUploadRecord.txt','a+')
        f.write(f'df_ftoffdir_{dateCSV}.csv, {len(df_ftoffdir)}, ftoffdir table, {len(ftoffdir_in_db)}\n')
        f.close()
    else:
        print("No new data")
        f = open('HotelOccupancyTaxData/formattedData/DBUploadRecord.txt','a+')
        f.write(f'df_ftoffdir_{dateCSV}.csv, {len(df_ftoffdir)}, ftoffdir table, {len(ftoffdir_in_db)}\n')
        f.close()
except Exception as e:
    print(f"Something went wrong, df_ftoffdir was not able to append to database: {e}")

No new data


## Call the tables within the database and store into a variable
* Going to compare new data from database with the df_fran and df_stp to filter

In [190]:
ftact_date_in_db = pdsql.read_sql("SELECT Taxpayer_Number FROM franchise_tax_info_date",engine)
stact_date_in_db = pdsql.read_sql("SELECT Taxpayer_Number FROM sales_tax_info_date",engine)
print(f"There are {len(ftact_date_in_db)} records in frachise tax permit date table.\nThere are {len(stact_date_in_db)} records in sales tax permit date table. ")

There are 101575 records in frachise tax permit date table.
There are 51946 records in sales tax permit date table. 


## fran aka df_fran
#### Checking table df with df data to make sure their are not duplicate tax paying numbers
* filtering new df_fran with data from the database
* Checking data for df_fran and also adding a new column of today's date and time
* Appending new companies (fran) to  csv and Database

In [191]:
try:
    df_fran = df_fran[~df_fran['Taxpayer_Number'].astype(int).isin(ftact_date_in_db['Taxpayer_Number'].astype(int))]
    if df_fran.size != 0:
        df_fran['DateTime'] = DateTimeSent
        print(f"There are {len(ftact_date_in_db)} data attributes in df_fran table from the database\n{len(df_fran)} new companies, based on tax payer number from filtered data df_fran")
        formattedData = os.path.abspath('HotelOccupancyTaxData/formattedData/'+ 'df_fran' + dateCSV + '.csv')
        df_fran.to_csv(formattedData, index = False)
        df_fran.to_sql(name='franchise_tax_info_date', con=engine, if_exists='append', index=False, chunksize=1000)
        print(f"df_fran to database append, completed")
        f = open('HotelOccupancyTaxData/formattedData/DBUploadRecord.txt','a+')
        f.write(f'fran_{dateCSV}.csv, {len(df_fran)}, franchise_tax_info_date table, {len(ftact_date_in_db)}\n')
        f.close()
    else:
        print("No new data")
        f = open('HotelOccupancyTaxData/formattedData/DBUploadRecord.txt','a+')
        f.write(f'fran_{dateCSV}.csv, {len(df_fran)}, franchise_tax_info_date table, {len(ftact_date_in_db)}\n')
        f.close()
except Exception as e:
    print(f"Something went wrong, df_fran was not able to append to database: {e}")

There are 101575 data attributes in df_fran table from the database
5518 new companies, based on tax payer number from filtered data df_fran
df_fran to database append, completed


<!-- #### Appending new companies (fran) to  csv and Database -->

## stp aka df_stp
#### Checking table df with df data to make sure their are not duplicate tax paying numbers
* filtering new df_stp with data from the database
* Checking data for df_stp and also adding a new column of today's date and time
* Appending new companies (stp) to  csv and Database

In [192]:
try:
    df_stp = df_stp[~df_stp['Taxpayer_Number'].astype(int).isin(stact_date_in_db['Taxpayer_Number'].astype(int))]
    if df_stp.size != 0:
        df_stp['DateTime'] = DateTimeSent
        print(f"There are {len(stact_date_in_db)} data attributes in df_stp table from the database\n{len(df_stp)} new companies, based on tax payer number from filtered data df_fran")
        formattedData = os.path.abspath('HotelOccupancyTaxData/formattedData/'+ 'df_stp' + dateCSV + '.csv')
        df_stp.to_csv(formattedData, index = False)
        df_stp.to_sql(name='sales_tax_info_date', con=engine, if_exists='append', index=False, chunksize=1000)
        print(f"df_stp to database append, completed")
        f = open('HotelOccupancyTaxData/formattedData/DBUploadRecord.txt','a+')
        f.write(f'stp_{dateCSV}.csv, {len(df_stp)}, sales_tax_info_date table, {len(stact_date_in_db)}\n---------------------end---------------------\n\n')
        f.close()
    else:
        print('No new data')
        f = open('HotelOccupancyTaxData/formattedData/DBUploadRecord.txt','a+')
        f.write(f'stp_{dateCSV}.csv, {len(df_stp)}, sales_tax_info_date table, {len(stact_date_in_db)}\n---------------------end---------------------\n\n')
        f.close()
except Exception as e:
    print(f"Something went wrong, df_stp was not able to append to database: {e}")

There are 51946 data attributes in df_stp table from the database
2976 new companies, based on tax payer number from filtered data df_fran
df_stp to database append, completed
